# Pendientes

* HECHO - convertir control en diccionario
* script mas complejo para proporcion
    - puede haber nulos, solucionar los faltantes
    - poder darle una lista de meses
* HECHO - arreglar scripts para control
* reescribir optimizacion de hiperparam
* arreglar teste de perdida al final

In [37]:
from datetime import datetime
import pandas as pd
import sys
import json
import os

# Archivos:
Luego del original, sell_in q es el source con millones de datos

- "group": solo numero de grupo, periodo, y tb. Punto de partida para las predicciones
- "prod_stats": identificador por grupo, media, maximos, minimos y otras estadisticas
- "prod_data_group": informacion sobre los productos, agrupado nivel grupo (groupby cats, descripcion etc)
- "prod_data_ungroup": informacion sobre los productos original, intacto del source
- "stock_data": info de stocks, intacto del source
- "norm": la informacion de "group" normalizada por media y desvio, columna tn_norm
- "lag": sigue luego de norm, mismas filas pero incorporando todo el feature engineering (no solo lags)
- "train": set de datos para train a partir de "lag" con los meses segun config
- "test": set de datos para test a partir de "lag" con los meses segun config
- "futuro": set de datos para futuro a partir de "lag" con los meses segun config
- "pred_test": predicciones de test, con algunas columnas accesorias
- "pred_futuro": predicciones de futuro, con algunas columnas accesorias
- "kaggle": prediccion en formato kaggle para el submit

In [38]:
ejecutar_scripts = {
    '01_LecturaDatos' : False,
    '02_normaliza': False,
    '02b_DTW' : False,
    '03_features' : False,
    '04_crear_train_set' : False,
    '05_lightgbm' : True,
    '06_evaluar_prediccion' : True	
}

In [39]:
NOMBRE_EXPERIMENTO = '2024-06-27_NivelProducto'
FOLDER = f'./Experimentos/{NOMBRE_EXPERIMENTO}/'

dibujar_pesos = False
lgbm_params = {
    'objective': 'regression',
    'metric': ['l2', 'rmse'],
    'boosting_type': 'gbdt',
        'bagging_fraction': 0.9,
        'bagging_freq': 1,
    'num_leaves': 40,
    'max_depth': -1,
    #'n_estimators': 50, 
    'feature_fraction': 0.9,
    'max_bin': 1023,
    'learning_rate': 0.25,
    'force_col_wise' : True,
    'num_threads' : 8,
    'verbose' : 1,
    'early_stopping_rounds' : 10
}

gen_config = {
    "nombre_exp" : NOMBRE_EXPERIMENTO,
    "folder" : FOLDER,
    "path_group" : "01_group.parquet",
    "path_prod_stats" : "01_prod_stats.parquet",
    "path_prod_data" : "01_prod_data.parquet",
    "path_stock_data" : "01_stock_data.parquet",
    "path_norm" : "02_norm.parquet",
    "path_dtw" : "02b_dtw.parquet",
    "path_traindtw" : "02b_traindtw.npy",
    "path_fitdtw" : "02b_fitdtw.npy",
    "path_dtw_model" : "02b_dtw_model.pkl",
    "path_lag" : "03_lag.parquet",
    "path_train" : "04_train.parquet",
    "path_test" : "04_test.parquet",
    "path_futuro" : "04_futuro.parquet",
    "path_pred_test" : "05_pred_test.parquet",
    "path_pred_futuro" : "05_pred_futuro.parquet",
    "path_kaggle" : f"kaggle_{NOMBRE_EXPERIMENTO}.csv",

    #"var_periodo_rep_start" : 201912,
    #"var_periodo_rep_end" : 201912,
    "var_ejecutar_dtw": True,
    "var_clusters": [10],
    "var_dibujar_dtw": False, #no poner true si no se rehacen los modelos
    "var_corte_prod_dtw" : 20100,
    "var_leer_pickle_dtw" : True,
    "var_lags": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 18, 21, 24, 30, 36],
    "var_cates_feat" : ['cat1', 'cat2', 'brand', 'descripcion'],
    "var_exclusiones" : [],

    "var_periodo_train_start" : 201701,
    "var_periodo_train_end" : 201910,
    "var_periodo_test" : 201910,
    "var_periodo_futuro" : 201912,

    "var_num_boost_round": 5000,
    "var_lgbm_params" : lgbm_params,
    "var_dibujar_pesos" : dibujar_pesos,
}

with open('gen_config.json', 'w') as file:
    json.dump(gen_config, file, indent=4)

if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)
with open(f'{FOLDER}\gen_config.json', 'w') as file:
    json.dump(gen_config, file, indent=4)

In [40]:
original_stdout = sys.stdout 

with open(f'{FOLDER}\exp_config_out.txt', 'w') as file:
    sys.stdout = file

    print(f"{'Experimento':.^120}\n{gen_config['nombre_exp']:-^120}")
    print(f"{'CARPETA':.<25}: {gen_config['folder']}")
    for k, v in gen_config.items():
        if k == 'folder' or k == 'nombre_exp':
            continue
        if k.startswith('path_'):
            print(f"{k:.<25}: {v} ( {FOLDER}{v} )")
        if k.startswith('var_'):
            print(f"{k:.<25}: {v}")

sys.stdout = original_stdout

with open(f'{FOLDER}\exp_config_out.txt', 'r') as file:
    print(file.read())

......................................................Experimento.......................................................
------------------------------------------------2024-06-27_NivelProducto------------------------------------------------
CARPETA..................: ./Experimentos/2024-06-27_NivelProducto/
path_group...............: 01_group.parquet ( ./Experimentos/2024-06-27_NivelProducto/01_group.parquet )
path_prod_stats..........: 01_prod_stats.parquet ( ./Experimentos/2024-06-27_NivelProducto/01_prod_stats.parquet )
path_prod_data...........: 01_prod_data.parquet ( ./Experimentos/2024-06-27_NivelProducto/01_prod_data.parquet )
path_stock_data..........: 01_stock_data.parquet ( ./Experimentos/2024-06-27_NivelProducto/01_stock_data.parquet )
path_norm................: 02_norm.parquet ( ./Experimentos/2024-06-27_NivelProducto/02_norm.parquet )
path_dtw.................: 02b_dtw.parquet ( ./Experimentos/2024-06-27_NivelProducto/02b_dtw.parquet )
path_traindtw............: 02b_train

In [41]:
if ejecutar_scripts['01_LecturaDatos']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 01_LecturaDatos.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'01_LecturaDatos':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")

if ejecutar_scripts['02_normaliza']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 02_normaliza.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'02_normaliza':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")

if ejecutar_scripts['02b_DTW']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 02b_DTW.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'02b_DTW':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")

if ejecutar_scripts['03_features']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 03_features.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'03_features':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")

if ejecutar_scripts['04_crear_train_set']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 04_crear_train_set.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'04_crear_train_set':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")

if ejecutar_scripts['05_lightgbm']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 05_lightgbm.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'05_lightgbm':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")

if ejecutar_scripts['06_evaluar_prediccion']:
    print(f"a las {datetime.now().strftime('%H:%M:%S')}")
    %run 06_evaluar_prediccion.ipynb
else:
    print(f"{'SALTEADO':-^100}")
    print(f"{'06_evaluar_prediccion':-^100}")
    print(f"{'SALTEADO':-^100}\n\n\n")


----------------------------------------------SALTEADO----------------------------------------------
------------------------------------------01_LecturaDatos-------------------------------------------
----------------------------------------------SALTEADO----------------------------------------------



----------------------------------------------SALTEADO----------------------------------------------
--------------------------------------------02_normaliza--------------------------------------------
----------------------------------------------SALTEADO----------------------------------------------



----------------------------------------------SALTEADO----------------------------------------------
----------------------------------------------02b_DTW-----------------------------------------------
----------------------------------------------SALTEADO----------------------------------------------



----------------------------------------------SALTEADO----------------------------

Shape df_train...........: (2173865, 188)
Shape df_test............: (68823, 188)
Shape df_futuro..........: (53008, 188)
Nulos en tn_futuro de Test: 51079
Shape df_test dropna.....: (17744, 188)
DISTRIBUCION DE DATOS EN CLUSTERS:
          train  train_prop  test  test_prop  futuro  futuro_prop
cluster                                                          
0        179653    0.082642  2212   0.124662    4356     0.082176
1        174193    0.080131   396   0.022317    5424     0.102324
2        250628    0.115291  2002   0.112827    5358     0.101079
3        317325    0.145973  3928   0.221371    7276     0.137262
4        291646    0.134160  1159   0.065318    6279     0.118454
5        185434    0.085302  1543   0.086959    5119     0.096570
6        136073    0.062595  1818   0.102457    3082     0.058142
7        148670    0.068390   586   0.033025    4580     0.086402
8        186891    0.085972  2907   0.163830    4402     0.083044
9        303352    0.139545  1193   0.06723